# SimilarWebの数値取得

## ライブラリの呼び出し

In [3]:
import sys,json
import urllib.request
import os
from dotenv import load_dotenv, find_dotenv

## 環境変数の呼び出し

In [4]:
#  ディレクトリから.envファイルを探す
dotenv_path = find_dotenv()

# 環境変数をロード
load_dotenv(dotenv_path)

# API_KEYに環境変数を代入
API_KEY = os.environ.get("SimilarWeb_API_KEY")

## APIコール用のURL作成

In [9]:
version = 'v1'
domain = 'employment.en-japan.com'
endpoint_group = 'traffic-and-engagement'
endpoint = 'visits'
start_date = '2017-09'
end_date = '2017-09'

url_format = 'https://api.similarweb.com/{}/website/{}/{}/{}?api_key={}&start_date={}&end_date={}&main_domain_only=false&granularity=monthly&country=JP'
url = url_format.format(version,domain,endpoint_group,endpoint,API_KEY,start_date,end_date)

## レスポンスを取得

In [12]:
# APIコールし、HTTPレスポンスをresponseに格納
response = urllib.request.urlopen(url)

# HTTPレスポンスをjsonにパース
read = response.read()
data = json.loads(read)

In [13]:
data

{'meta': {'device': 'Desktop',
  'last_updated': '2017-12-31',
  'request': {'country': 'JP',
   'domain': 'employment.en-japan.com',
   'end_date': '2017-09-30',
   'format': None,
   'granularity': 'Monthly',
   'limit': None,
   'main_domain_only': False,
   'start_date': '2017-09-01'},
  'status': 'Success'},
 'visits': [{'date': '2017-09-01', 'visits': 2559298.422281931}]}